### Instructions
A - above
B - below
X - delete 
Ctrl Enter - run
Enter - Enter into cell
M - make cell into markdown cell

In [76]:
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [117]:
f = open("modules.txt", 'r')
#print(f.read())
f.close()

# THIS IS A HEADING. ITS NEAT.
# BELOW IT IS SOME CODE.

In [116]:
import numpy as np

def generate_timetable():
    """
    Generate a timetable Spyx
    Read the file to see how many classes + labs there are.
    Store them in a 20x3 list alongside empty sessions to fill the list
    """
    #Create "empty" timetable
    timetable = [["empty" for x in range(3)] for y in range(20)] 
    
    #Read file into allSplits list
    allSplits = []
    f = open("modules.txt", 'r')
    lineCount = 0
    for line in f:
        allSplits.append(line.split("|"))
        lineCount+=1
   
    #Add module into timetable
    for i in range(0,lineCount):
        timetable[i][0] = allSplits[i][0]
    
    
    #Add seminars into timetable.
    #Iterate through each module, for each number of seminars use
    #counters to iterate to next empty position and place seminar.
    #Should work on different set of modules if format is kept identical.
    emptyChecker = 0
    emptyChecker1 = 0
    for i in range(0, lineCount):
        for j in range(0, int(allSplits[i][2])):
            toggle = False
            while toggle == False:
                if timetable[emptyChecker][emptyChecker1] == "empty":
                    timetable[emptyChecker][emptyChecker1] = allSplits[i][0]
                    toggle = True
                emptyChecker1+=1
                if emptyChecker1 == 3:
                    emptyChecker+=1
                    emptyChecker1=0
    
    #randomises through randomise function below
    timetable = randomise_timetable(timetable)
    
    #for i in range(0, len(timetable)):
        #print(timetable[i])
    #print(timetableJoined)
    
    return timetable


In [104]:
def randomise_timetable(timetable):
    #Create randomised timetable
    #Add all to single list
    #Shuffle list
    #Put back into timetable
    
    timetableJoined = []
    for list in timetable:
        for i in list:
            timetableJoined.append(i)
    np.random.shuffle(timetableJoined)
    for i in range(0,len(timetable)):
        for j in range(0,2):
            timetable[i][j] = timetableJoined[i]
    
    return timetable

In [ ]:
def room_validity(timetable):
    #Check to see if timetable is valid ie:
    #lectures take place in lecture theatre & labs are in labs
    #returns true if valid, else false if not
    
    

In [100]:
def fitness_function():
    """
    Generate a fitness calculator Spyx
    allSplits = []
    f = open("modules.txt", 'r')
    for line in f:
        allSplits.append(line.split("|"))
    print(allSplits[1][0])
    """
    generate_timetable()
    
    
    """
    CONSTRAINTS:
    1) ONLY 1 LECTURE AND TWO LABS MAY RUN CONCURRENTLY
    2) LABS FOR A MODULE MAY ONLY OCCUR AFTER ITS LECTURE! PRECEDENCE!
    3) SESSIONS CAN'T INTERFERE! CONCURRENCE!
    """
    
fitness_function()
    

MOD002
